In [ ]:
import gdown
import zipfile
from glob import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.ensemble import RandomForestRegressor
import csv
import json
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore

In [ ]:
# Download base de dados
file_id = '1JcR3KRQDwJyzqTbCiYF9IHRLYd0_8ITC'
url = f'https://drive.google.com/uc?id={file_id}'

output = 'open-data.zip'
gdown.download(url, output, quiet=False)

with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall('/content/')

Downloading...
From (original): https://drive.google.com/uc?id=1JcR3KRQDwJyzqTbCiYF9IHRLYd0_8ITC
From (redirected): https://drive.google.com/uc?id=1JcR3KRQDwJyzqTbCiYF9IHRLYd0_8ITC&confirm=t&uuid=b9e5a8bd-e719-4a97-8d24-0fa80f07bc54
To: /content/open-data.zip
100%|██████████| 42.1M/42.1M [00:00<00:00, 65.1MB/s]


## Pré-processamento


In [ ]:
clientes = glob("/content/Train/dash/*")

In [ ]:
clientes_servidores = {'ba': 0, 'rj': 1, 'ce': 0, 'df': 1, 'es': 2, 'pi': 3}

In [ ]:
X = []
y = []

for cliente in clientes:
  servidores = glob(os.path.join(cliente, '*'))

  for servidor in servidores:
    requisicoes = servidores = glob(os.path.join(servidor, '*'))

    dash_values = []
    for file_path in requisicoes:
      with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines[:-1]:
            data = json.loads(line)
            dash_values.append({
                'timestamp': pd.to_datetime(data['timestamp'], unit='s'),
                'rate': data['rate']
            })

    dash_serie = pd.DataFrame(dash_values)
    dash_serie.set_index("timestamp", inplace=True)
    dash_serie = dash_serie.sort_index()

    dash_serie_5min = dash_serie.resample('5min').agg({'rate': ['mean', 'std']}).dropna()

    for i in range(len(dash_serie_5min) - 12):
        group = dash_serie_5min.iloc[i : i + 12]

        X.append([])
        y.append([])

        X[-1].append(clientes_servidores[cliente.split('/')[-1]]) #cliente
        X[-1].append(clientes_servidores[servidor.split('/')[-1]]) #servidor
        X[-1].append(group[('rate', 'mean')].iloc[0:10].mean())
        X[-1].append(group[('rate', 'std')].iloc[0:10].std())
        X[-1].append(group[('rate', 'mean')].iloc[9])
        X[-1].append(group[('rate', 'std')].iloc[9])

        y[-1].append(group[('rate', 'mean')].iloc[10])
        y[-1].append(group[('rate', 'std')].iloc[10])
        y[-1].append(group[('rate', 'mean')].iloc[11])
        y[-1].append(group[('rate', 'std')].iloc[11])

In [ ]:
X_np = np.array(X)
y_np = np.array(y)

In [ ]:
X_train = X_np
y_train = y_np

#Treinamento

In [ ]:
rf_regressor = RandomForestRegressor(n_estimators=600, max_features= 0.5, max_depth=20, min_samples_split=2, min_samples_leaf=20, random_state=42)

# Teste

In [ ]:
features = []
submission = []

In [ ]:
Test_jsons = glob("/content/Test/*")

In [ ]:
for test in Test_jsons:
  with open(test, 'r') as file:
    data = json.load(file)

    submission.append([])
    submission[-1].append(test.split('/')[-1].split('.')[0])

    rates_mean = []
    rates_std = []
    for dash in data['dash']:
      rates_mean.append(np.array(dash['rate']).mean())
      rates_std.append(np.array(dash['rate']).std())

    features.append([])
    features[-1].append(clientes_servidores[data['cliente']])
    features[-1].append(clientes_servidores[data['servidor']])
    features[-1].append(np.array(rates_mean).mean())
    features[-1].append(np.array(rates_mean).std())
    features[-1].append(rates_mean[-1])
    features[-1].append(rates_std[-1])

In [ ]:
features_np = np.array(features)

In [ ]:
for i in range(4):
  rf_regressor.fit(X_np, y_np[:, i])
  y_pred = rf_regressor.predict(features_np)

  for j, y_pred_value in enumerate(y_pred):
    submission[j].append(y_pred_value)

In [ ]:
header = ["id", "mean_1", "stdev_1", "mean_2", "stdev_2"] #eu add melissa, pra n ter q fazer manual

with open("Algumacoisa1.csv", mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(header)
    writer.writerows(submission)

print(f"Arquivo foi criado com sucesso!")

Arquivo foi criado com sucesso!
